In [2]:
!echo Installing bbSearch module from web ...
!echo creating bbmodcache subfolder
!mkdir -p bbmodcache
!echo downloading bbSearch module
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py

from bbmodcache.bbSearch import SearchProblem, search

Installing bbSearch module from web ...
creating bbmodcache subfolder
downloading bbSearch module
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18767  100 18767    0     0   407k      0 --:--:-- --:--:-- --:--:--  416k
Loading bbSearch Version 2.1 (at 11:23, Fri 21 Feb)
Last module source code edit 9am Thursday 24th Feb 2022


In [ ]:
from copy import deepcopy
from datetime import datetime, timedelta
import random

# Room positions in a 3x3 grid
ROOM_POSITIONS = {
    "History": (0, 0), "English": (0, 1), "Maths": (0, 2),
    "Music": (1, 0), "Storage": (1, 1), "Drama": (1, 2),
    "Physics": (2, 0), "Business": (2, 1), "Office": (2, 2)
}

# Item weights (assuming all books weigh 1 unit)
ITEM_WEIGHT = { "book_History": 1, "book_English": 1, "book_Maths": 1, 
                "book_Music": 1, "book_Drama": 1, "book_Physics": 1, 
                "book_Business": 1 }

class RobotState:
    def __init__(self, location, carried_items, room_contents, keys):
        self.robot = self
        self.location = location
        self.carried_items = carried_items  # Books & keycards
        self.room_contents = room_contents  # Books in each room
        self.keys = keys  # Keys the robot has

    def weight_carried(self):
        return sum(ITEM_WEIGHT.get(i, 0) for i in self.carried_items)

    def __eq__(self, other):
        return (self.location == other.location and 
                self.carried_items == other.carried_items and 
                self.room_contents == other.room_contents and 
                self.keys == other.keys)

    def __hash__(self):
        return hash((self.location, tuple(self.carried_items), tuple(sorted(self.keys))))

class RobotWorker(SearchProblem):
    def __init__(self, initial_state, goal_item_locations):
        self.initial_state = initial_state
        self.goal_item_locations = goal_item_locations

    def possible_actions(self, state):
        robot_location = state.robot.location
        actions = []

        # Put down any carried book
        for item in state.robot.carried_items:
            actions.append(("put down", item))

        # Pick up books if within carrying capacity
        for item in state.room_contents[robot_location]:
            if item.startswith("book_") and state.weight_carried() + ITEM_WEIGHT[item] <= 5:
                actions.append(("pick up", item))

        # Pick up keycards if in Office
        if robot_location == "Office":
            if "keycard_Maths" not in state.keys:
                actions.append(("pick up", "keycard_Maths"))
            if "keycard_English" not in state.keys:
                actions.append(("pick up", "keycard_English"))

        # Move to adjacent rooms (obeying locked room rules)
        for room, pos in ROOM_POSITIONS.items():
            if room != robot_location and self.is_adjacent(robot_location, room):
                if room in ["Maths", "English"]:
                    if f"keycard_{room}" in state.keys:  # Only enter if keycard is held
                        actions.append(("move to", room))
                else:
                    actions.append(("move to", room))

        return actions

    def successor(self, state, action):
        next_state = deepcopy(state)
        act, target = action

        if act == "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)

        if act == "pick up":
            if target.startswith("keycard_"):
                next_state.keys.add(target)  # Pick up keycard
            else:
                next_state.robot.carried_items.append(target)  # Pick up book
                next_state.room_contents[state.robot.location].remove(target)

        if act == "move to":
            next_state.robot.location = target

        return next_state

    def goal_test(self, state):
        # Books should be in their correct rooms
        for room, correct_books in self.goal_item_locations.items():
            for book in correct_books:
                if book not in state.room_contents[room]:
                    return False
        return True

    def is_adjacent(self, room1, room2):
        """Check if two rooms are adjacent in the 3x3 grid."""
        x1, y1 = ROOM_POSITIONS[room1]
        x2, y2 = ROOM_POSITIONS[room2]
        return abs(x1 - x2) + abs(y1 - y2) == 1  # Manhattan Distance == 1

    # def heuristic(self, state):
    #     """Estimate cost: books misplaced + distance + keycard penalty."""
    #     cost = 0

    #     for room, correct_books in self.goal_item_locations.items():
    #         for book in correct_books:
    #             if book not in state.room_contents[room]:  
    #                 book_location = next(
    #                     (r for r, items in state.room_contents.items() if book in items), None)
    #                 if book_location:
    #                     cost += abs(ROOM_POSITIONS[book_location][0] - ROOM_POSITIONS[room][0]) + \
    #                             abs(ROOM_POSITIONS[book_location][1] - ROOM_POSITIONS[room][1])

    #     # Keycard penalty
    #     if "Maths" in self.goal_item_locations and "keycard_Maths" not in state.keys:
    #         cost += 5
    #     if "English" in self.goal_item_locations and "keycard_English" not in state.keys:
    #         cost += 5

    #     return cost

    def display_state(self, state):
        print(f"Robot Location: {state.robot.location}")
        print(f"Carrying: {state.robot.carried_items}")
        print(f"Keys: {state.keys}")
        print("Room Contents:")
        for room, contents in state.room_contents.items():
            print(f"  {room}: {contents}")

# ---- INITIAL STATE ----
room_contents = {
    "Storage": {"book_History", "book_English", "book_Maths", "book_Music",
                "book_Drama", "book_Physics", "book_Business"},
    "Office": set(), "Maths": set(), "English": set(),
    "History": set(), "Music": set(), "Drama": set(),
    "Physics": set(), "Business": set()
}

robot_state = RobotState(location="Storage", carried_items=[], room_contents=room_contents, keys=set())

# ---- GOAL STATE ----
goal_items = {
    "History": {"book_History"},
    "English": {"book_English"},
    "Maths": {"book_Maths"},
    "Music": {"book_Music"},
    "Drama": {"book_Drama"},
    "Physics": {"book_Physics"},
    "Business": {"book_Business"},
}

# ---- SOLVING ----
robot_problem = RobotWorker(robot_state, goal_items)

max_nodes = 1000000  # Example value for max_nodes

print("BFS Solution:")
search(robot_problem, 'BF/FIFO', max_nodes)  # Correct mode for BFS

print("\nDFS Solution:")
search(robot_problem, 'DF/LIFO', max_nodes)  # Correct mode for DFS



BFS Solution:
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 1000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)


.........................